# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298493491875                   -0.85    4.1
  2   -8.300203157154       -2.77       -1.25    1.5
  3   -8.300451228308       -3.61       -1.87    3.6
  4   -8.300462786245       -4.94       -2.73    2.4
  5   -8.300464373656       -5.80       -3.14    5.2
  6   -8.300464504748       -6.88       -3.27    5.4
  7   -8.300464571712       -7.17       -3.41    1.6
  8   -8.300464606789       -7.45       -3.53    1.8
  9   -8.300464636860       -7.52       -3.76    1.0
 10   -8.300464638562       -8.77       -3.83    2.4
 11   -8.300464643736       -8.29       -4.21    1.8


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64192241416                   -0.70    5.4
  2   -16.67865487127       -1.43       -1.14    1.8
  3   -16.67922505042       -3.24       -1.87    3.4
  4   -16.67927941431       -4.26       -2.75    4.8
  5   -16.67928594178       -5.19       -3.12    4.2
  6   -16.67928621002       -6.57       -3.52    1.5
  7   -16.67928622141       -7.94       -4.02    2.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32472731593                   -0.56    6.5
  2   -33.33262154810       -2.10       -0.99    1.0
  3   -33.33592376456       -2.48       -1.69    4.4
  4   -33.33687157288       -3.02       -2.35    4.1
  5   -33.33693754455       -4.18       -2.85   11.0
  6   -33.33694384609       -5.20       -3.74    2.4
  7   -33.33694391447       -7.17       -4.14    9.1


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298668241238                   -0.85    4.1
  2   -8.300301274728       -2.79       -1.59    1.0
  3   -8.300458443766       -3.80       -2.63    7.0
  4   -8.300420586146   +   -4.42       -2.43   10.0
  5   -8.300464527628       -4.36       -3.72    2.4
  6   -8.300464620925       -7.03       -4.13    4.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32578421594                   -0.56    6.5
  2   -33.32683089219       -2.98       -1.27    1.1
  3   -22.96794221841   +    1.02       -0.56    7.2
  4   -33.25831542796        1.01       -1.60    8.8
  5   -33.11934818740   +   -0.86       -1.25    4.1
  6   -32.44393833034   +   -0.17       -1.09    4.6
  7   -33.32702158891       -0.05       -1.99    4.8
  8   -33.33422875068       -2.14       -2.29    2.4
  9   -33.33557469494       -2.87       -2.41    2.4
 10   -33.33558188970       -5.14       -2.46    1.4
 11   -33.33632955114       -3.13       -2.61    1.8
 12   -33.33691945357       -3.23       -3.21    3.1
 13   -33.33693882685       -4.71       -3.39    3.4
 14   -33.33694252975       -5.43       -3.56    2.1
 15   -33.33694114201   +   -5.86       -3.79    3.4
 16   -33.33693957261   +   -5.80       -3.73    3.4
 17   -33.33694379895       -5.37       -4.29 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.